In [ ]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import matplotlib.pyplot as plt # this is used for the plot the graph 
import tensorflow_probability as tfp
from tqdm import tnrange, tqdm_notebook
import seaborn as sns
import tensorflow as tf
from scipy import stats
tfd=tfp.distributions
from tqdm import notebook
from scipy import optimize
from functools import partial
from scipy.interpolate import interp1d
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")
tf.keras.backend.set_floatx('float64')

In [ ]:
%matplotlib inline

### Simulate Data

In [ ]:
np.random.seed(2020)

# generate weibull distribution parameter
shape=np.random.uniform(1,5,1000)
scale=np.random.uniform(0.5,2,1000)


# the full design matrix
x=np.c_[shape,scale]


y=(np.random.weibull(shape,size=1000)*scale).reshape(-1,1)


train_x=x[:700,:]
train_y=y[:700,:]

test_x=x[700:,:]
test_y=y[700:,:]

ntrain=len(train_x)
ntest=len(test_x)



#### normalization function

In [ ]:
##define a normalization function
def norm_data(train, test,norm=True):
    std_train=np.ones(train.shape[1])
    mu_train=np.zeros(train.shape[1])
    if(norm):
        std_train = np.std(train, 0)
        mu_train=np.mean(train,0)

    train=(train - np.full(train.shape, mu_train)) / \
            np.full(train.shape, std_train)
    test=(test - np.full(test.shape, mu_train)) / \
            np.full(test.shape, std_train)
    return train,test,mu_train,std_train

In [ ]:
train_x,test_x,_,_=norm_data(train_x,test_x,norm=False)
#train_y,_,muy,stdy=norm_data(train_y,test_y,norm=False)

### Ensemble method based of hetero regression(EN)

In [ ]:
class EN:
    def __init__(self,tau):
        self.tau=tau
    
    def model_musd(self,trainx,trainy,testx):
        #the model
        model=tf.keras.Sequential([
            tf.keras.layers.Dense(100,activation=tf.nn.elu),
            tf.keras.layers.BatchNormalization(axis=-1),
            tf.keras.layers.Dense(80,activation=tf.nn.elu),
            tf.keras.layers.BatchNormalization(axis=-1),
            tf.keras.layers.Dense(1+1),
            tfp.layers.DistributionLambda(
            lambda t:tfd.Normal(loc=t[...,:1],
            scale=tf.math.softplus(self.tau*t[...,1:])+0.001)),
        ])

        #the loss
        negloglik = lambda y, p_y: -p_y.log_prob(y)

        #the model fitting for 5 models
        mut=np.zeros(len(testx))
        sdt=np.zeros(len(testx))
        
       
        STEPS_PER_EPOCH=np.floor(len(trainx)/128)

        #learning rate    
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
          0.001,
          decay_steps=STEPS_PER_EPOCH*100,
          decay_rate=1,
          staircase=False)

        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        for i in notebook.tnrange(5):
            model.compile(optimizer=get_optimizer(),loss=negloglik)
            model.fit(trainx,trainy,epochs=300,batch_size=128,verbose=0)
            yhatt=model(testx)
            mu1=np.array(yhatt.mean()).ravel()
            sd1=np.array(yhatt.stddev()).ravel()
            mut=np.c_[mut,mu1]
            sdt=np.c_[sdt,sd1]

        sdt=sdt[:,1:]
        mut=mut[:,1:]


        #the averaged mean and sd for the ensemble model
        muhat=np.mean(mut,1)
        sighat=np.sqrt(np.mean(sdt**2+mut**2,1)-muhat**2)
        return muhat, sighat


### Single Evaluation, with tau set as 0.1

In [ ]:
ENmodel=EN(0.1)
enmu,enstd=ENmodel.model_musd(train_x,train_y,test_x)

#### P(Y>1|X)

In [ ]:
#true
tsuv1=1-stats.weibull_min.cdf(1,c=test_x[:,0],scale=test_x[:,1])

#cdf estimate by g
ensuv1=1.-(stats.norm.cdf((1.-enmu)/enstd))

plt.figure(figsize=(5,5))
plt.plot(tsuv1,ensuv1,'.')

plt.plot([0,1],[0,1])
#np.save('ensuv_est',ensuv1)

#### Recovery of true cdf

In [ ]:
#generate sample
np.random.seed(3421)
samps=np.random.choice(len(test_x),3)
yrange=xtmp=np.linspace(-2,7,5000) #gaussian can go below 0
## mean and sd for dp sample
enmusd=np.c_[enmu,enstd][samps]
enmusd

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(131)

plt.subplot(1,3,1)
i=samps[0]
tcdf=tcdf=stats.weibull_min.cdf(x=yrange,c=test_x[i,0],scale=test_x[i,1])
encdf=stats.norm.cdf((yrange-enmusd[0,0])/enmusd[0,1])

plt.plot(yrange,encdf)
plt.plot(yrange,tcdf)




plt.subplot(1,3,2)
i=samps[1]
tcdf=tcdf=stats.weibull_min.cdf(x=yrange,c=test_x[i,0],scale=test_x[i,1])
encdf=stats.norm.cdf((yrange-enmusd[1,0])/enmusd[1,1])

plt.plot(yrange,encdf)
plt.plot(yrange,tcdf)



plt.subplot(1,3,3)
i=samps[2]
tcdf=tcdf=stats.weibull_min.cdf(x=yrange,c=test_x[i,0],scale=test_x[i,1])
encdf=stats.norm.cdf((yrange-enmusd[2,0])/enmusd[2,1])

plt.plot(yrange,encdf)
plt.plot(yrange[1000:],tcdf[1000:])


### Ten replications to evaluate the hard metrics

In [ ]:
##function to create replication
def rep_iter(x,y,frac=0.3):
    n=len(x)
    ntest=int(np.floor(frac*n))
    allidx=np.random.permutation(n)
    trainidx= allidx[ntest:]
    testidx= allidx[:ntest]
    return x[trainidx],y[trainidx],x[testidx],y[testidx]
    

In [ ]:
#initialize the metric
enll=[]
encal=[]
en90=[]
enmae=[]

In [ ]:
np.random.seed(2021)
for a in range(10):
    train_x,train_y,test_x,test_y=rep_iter(x,y)
    ntrain=len(train_x)
    ntest=len(test_x)

    
    ENmodel=EN(0.1)
    enmu,enstd=ENmodel.model_musd(train_x,train_y,test_x)
    
    
    #####calculate metrics##############

    per=np.linspace(0.02,0.98,8) #quantile to study calibration

    enc=[]

    for i in per:
        lquantile=(stats.norm.ppf(0.5-i/2.)*enstd+enmu)
        rquantile=(stats.norm.ppf(0.5+i/2.)*enstd+enmu)
        enc.append(np.mean((test_y.ravel()<rquantile.ravel())*(test_y.ravel()>lquantile.ravel())))

    encal.append(np.abs(enc-per).mean())
    
    #ninty
    l90=(stats.norm.ppf(0.5-0.9/2.)*enstd+enmu)
    r90=(stats.norm.ppf(0.5+0.9/2.)*enstd+enmu)
    en90.append(np.mean((test_y.ravel()<r90.ravel())*(test_y.ravel()>l90.ravel())))
    
    #log likelihood
    low=np.quantile(test_y,0.05)
    high=np.quantile(test_y,0.95)
    itv=np.linspace(low,high,9)
    itv=np.append(-np.infty,itv)
    itv=np.append(itv,np.infty)
    #outcome1 belongs to which interval
    id=np.zeros(len(test_y))
    for i in range(10):
        id=id+1*(test_y.ravel()>itv[i+1])
    id=id.astype('int')

    l=(itv[id]-enmu)/enstd
    r=(itv[id+1]-enmu)/enstd
    prtmp=stats.norm.cdf(r)-stats.norm.cdf(l)
    ll_est=np.log(prtmp)
    enll.append(ll_est.mean())
    
    #mae
    enmae.append(np.abs(enmu-stats.weibull_min.ppf(0.5,c=test_x[:,0],scale=test_x[:,1])).mean())
    
    


In [ ]:
def musd(x):
    print(np.mean(x),np.std(x))

musd(enll)
musd(encal)
musd(en90)
musd(enmae)
